In [2]:
from unstructured.partition.pdf import partition_pdf

/Users/jshah/micromamba/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# pdf https://www.nerc.com/pa/RAPA/PA/Section1600DataRequestsDL/2024_GADS_Wind_DRI.pdf

pdf_path = "2024_GADS_Wind_DRI.pdf"
elements = partition_pdf(pdf_path, 
                strategy="hi_res",
                infer_table_structure=True,
                extract_images_in_pdf=True,
                max_partition=None)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
len(elements)

903

In [6]:
import json
import pandas as pd

elements_dict = [e.to_dict() for e in elements]
df = pd.DataFrame(elements_dict)
df.head()

,type,element_id,text,metadata
0,Image,389c6ad1643f9f82c5c623f784f06f46,GADS Wind...,{'coordinates': {'points': ((48.05555555555556...
1,Title,bdbfccd772321c20f8d1ac72b2f09c15,GADS Wind Turbine Generation Data Reporting In...,"{'coordinates': {'points': ((150.0, 587.988888..."
2,Title,18de5682df61164c41ade86197ea52c8,"Effective: January 1, 2024","{'coordinates': {'points': ((150.0, 1054.60555..."
3,Image,1d4789feb7bd6d90a4655dcf196facd4,RELIABILITY | RESILIENCE | SECURITY,{'coordinates': {'points': ((806.6666666666666...
4,Image,5f3ee18bb03cd212a1527e27eb064181,"— ' — . i, y \","{'coordinates': {'points': ((47.5, 1624.305555..."


In [8]:
df['type'].value_counts()

type
NarrativeText        270
ListItem             202
Title                191
Table                 61
UncategorizedText     60
Header                55
Formula               34
Image                 22
FigureCaption          8
Name: count, dtype: int64

In [9]:
df[df['type'] == 'Table']['metadata'].iloc[0]

{'detection_class_prob': 0.9014275074005127,
 'coordinates': {'points': ((147.2567901611328, 208.85279846191406),
   (147.2567901611328, 1696.571044921875),
   (1564.552490234375, 1696.571044921875),
   (1564.552490234375, 208.85279846191406)),
  'system': 'PixelSpace',
  'layout_width': 1700,
  'layout_height': 2200},
 'last_modified': '2024-08-28T21:02:27',
 'text_as_html': '<table><tbody><tr><td colspan="2">EVISION HiStOLY...... ce ceceseseceseeseeseesseeeeesecsecseeesecsesseseaeeseecaecsecesecsasseeesseaeessecsecsuecseceseeecadsesesseseseaesaeeeaeseesesesseaseeeeaeesgee iv</td></tr><tr><td colspan="2">FOFACE oo. eececeeecesceceseescsecseeaeseeseacsecsecsseacsecseacsecsesaesacsesaesasseeaseassesaesaesaeseeaeseseesaesassecaeaeseeseeaeeeseesaeeessesaeeaseeeaeeaeeeeaeeatee \\</td></tr><tr><td colspan="2">‘troduction ....</td></tr><tr><td>WhO MUSt REPOFt.....cccccscesecsscsscesccsecssecsecseceseesesessesecesesecseceseseceesesecsecessesecseseseceesesseaeeesseseceeseeeseeseseaeseesesesees

In [10]:
# Convert a single extracted element of type 'Table' to a Markdown table format using 'text_as_html'
def convert_single_table_element_to_markdown(element):
    # Check if the element has 'text_as_html' in its metadata
    if 'text_as_html' not in element:
        raise ValueError("Provided element does not contain 'text_as_html'.")

    # Extract the HTML table from the element
    html_table = element['text_as_html']

    # Use BeautifulSoup to parse the HTML and convert it to Markdown
    from bs4 import BeautifulSoup

    # Parse the HTML
    soup = BeautifulSoup(html_table, 'html.parser')
    markdown_table = ""

    # Extract table headers
    headers = [th.get_text() for th in soup.find_all('th')]
    if headers:
        markdown_table += "| " + " | ".join(headers) + " |\n"
        markdown_table += "| " + " | ".join(['---'] * len(headers)) + " |\n"

    # Extract table rows
    for row in soup.find_all('tr'):
        cells = row.find_all(['td', 'th'])
        if cells:
            markdown_table += "| " + " | ".join(cell.get_text() for cell in cells) + " |\n"

    return markdown_table

# Example usage with a single table element
# markdown_output = convert_single_table_element_to_markdown(single_table_element)
# print(markdown_output)





In [11]:
convert_single_table_element_to_markdown(df[df['type'] == 'Table']['metadata'].iloc[0])

'| EVISION HiStOLY...... ce ceceseseceseeseeseesseeeeesecsecseeesecsesseseaeeseecaecsecesecsasseeesseaeessecsecsuecseceseeecadsesesseseseaesaeeeaeseesesesseaseeeeaeesgee iv |\n| FOFACE oo. eececeeecesceceseescsecseeaeseeseacsecsecsseacsecseacsecsesaesacsesaesasseeaseassesaesaesaeseeaeseseesaesassecaeaeseeseeaeeeseesaeeessesaeeaseeeaeeaeeeeaeeatee \\ |\n| ‘troduction .... |\n| WhO MUSt REPOFt.....cccccscesecsscsscesccsecssecsecseceseesesessesecesesecseceseseceesesecsecessesecseseseceesesseaeeesseseceeseeeseeseseaeseeseseseesesesaeeees | vi |\n| hapter 1: Date Record Types and Forimat.........cccecsscescssesseseeeesseeeeecseesceecseeaeseeseeaeeecsesaeaeseeseeaeseseesaeeessesaeeasesaeeeseeeaeeaees 1 |\n| Data RECOrd Type@s.......eessssscessscsscesceeseeseseesesecsecseesesecseaesessesaeacsecaesacsecseaeseeseeecsecsesaesaesaeseeaesesaesaseessesseeasesseeeseesaeeaeee 1 |\n| FOrMat ....c cee ccccceesecsescsecessceessessceecsecscsecsessesecsessessceecsecsssecssseassssessesessessusesseseessases

In [ ]:
# Function to convert a DataFrame of tables to Markdown format
def convert_dataframe_to_markdown(df):
    markdown_output = ""
    for index, row in df.iterrows():
        if row['type'] == 'Table':
            markdown_output += convert_single_table_element_to_markdown(row['metadata']) + "\n\n" + str(row['metadata']['page_number']) +"\n\n"
            
    return markdown_output

# Example usage with the DataFrame containing multiple elements
markdown_tables = convert_dataframe_to_markdown(df)
print(markdown_tables)


In [5]:
import requests

# Replace with your actual Modal Labs FastAPI endpoint URL
url = "https://jayshah5696--modalpdfprocessor-fastapi-app-dev.modal.run/process-pdf/"

# Path to the PDF file on your local machine
pdf_file_path = "2024_GADS_Wind_DRI.pdf"

# Open the PDF file in binary mode
def upload_pdf(endpoint_url, file_path):
    """Uploads a PDF file to the specified endpoint URL."""
    files = {'file': open(file_path, 'rb')}
    response = requests.post(endpoint_url, files=files,verify=False)
    if response.status_code == 200:
        print("PDF uploaded successfully!")
        return response.json()
    else:
        print(f"Error uploading PDF: {response.status_code}")
        print(response.text)
        return None



In [6]:
response = upload_pdf(url, pdf_file_path)

/Users/jshah/micromamba/envs/pytorch/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jayshah5696--modalpdfprocessor-fastapi-app-dev.modal.run'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jshah/micromamba/envs/pytorch/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jayshah5696--modalpdfprocessor-fastapi-app-dev.modal.run'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jshah/micromamba/envs/pytorch/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jayshah5696--modalpdfprocessor-fastapi-app-dev.modal.run'. Adding certific

PDF uploaded successfully!


In [11]:
import json
# save this as json
with open('response.json', 'w') as f:
    json.dump(response, f)

In [1]:
import json
import pandas as pd
response = json.load(open('response.json'))

In [2]:
from pandas import json_normalize

df = json_normalize(response)

In [3]:
df.head()

,type,text,element_id,metadata.coordinates.points,metadata.coordinates.system,metadata.coordinates.layout_width,metadata.coordinates.layout_height,metadata.filetype,metadata.languages,metadata.page_number,metadata.image_path,metadata.detection_class_prob,metadata.text_as_html,metadata.parent_id
0,Image,GADS Wind...,6cefe70712d1f985da24f15644359efb,"[[48.05555555555556, 53.6111111111113], [48.05...",PixelSpace,1700,2200,application/pdf,[eng],1,/root/figures/figure-1-1.jpg,NaN,NaN,NaN
1,Title,GADS Wind Turbine Generation Data Reporting In...,5427b6f5e20491efb96a5503dd1b5488,"[[150.0, 587.9888888888889], [150.0, 968.27222...",PixelSpace,1700,2200,application/pdf,[eng],1,NaN,NaN,NaN,NaN
2,Title,"Effective: January 1, 2024",8b22337286abf8536ed2c3daf0adb94b,"[[150.0, 1054.6055555555556], [150.0, 1121.272...",PixelSpace,1700,2200,application/pdf,[eng],1,NaN,NaN,NaN,NaN
3,Image,RELIABILITY | RESILIENCE | SECURITY,cc5f020eab26832b0efabe91565cd4db,"[[806.6666666666666, 1565.2222222222222], [806...",PixelSpace,1700,2200,application/pdf,[eng],1,/root/figures/figure-1-2.jpg,NaN,NaN,NaN
4,Image,"ee q — s i, y",10b53442f27c9dae8fbb6e904629ed52,"[[47.5, 1624.3055555555554], [47.5, 1926.38888...",PixelSpace,1700,2200,application/pdf,[eng],1,/root/figures/figure-1-3.jpg,NaN,NaN,NaN


In [4]:
df.columns

Index(['type', 'text', 'element_id', 'metadata.coordinates.points',
       'metadata.coordinates.system', 'metadata.coordinates.layout_width',
       'metadata.coordinates.layout_height', 'metadata.filetype',
       'metadata.languages', 'metadata.page_number', 'metadata.image_path',
       'metadata.detection_class_prob', 'metadata.text_as_html',
       'metadata.parent_id'],
      dtype='object')

In [5]:
df['text']

0                                          GADS  Wind...
1      GADS Wind Turbine Generation Data Reporting In...
2                             Effective: January 1, 2024
3                    RELIABILITY | RESILIENCE | SECURITY
4                                          ee q — s i, y
                             ...                        
896    Outage delayed by pandemic loss of manpower Wo...
897    Outage delayed by pandemic loss of manpower  W...
898                                               Time  
899                                                 Time
900    NERC | GADS Wind Data Reporting Instructions |...
Name: text, Length: 901, dtype: object

In [6]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")

df['text_encoded'] = df['text'].apply(lambda x: enc.decode(x,disallowed_special=()))

TypeError: argument 'tokens': Can't extract `str` to `Vec`